# Convert skymap files

## Overview

In this notebook, we will provide brief overviews of:
- what skymaps are
- why we need a way to convert their structure/representation
- how they're currently stuctured in lsst.skymaps
- a couple different strategies we use for this conversion

### What is a skymap?

(a brief explanation)

(maybe also describe how lsst structures/stores skymaps)

For more information on skymaps, see the Appendix 3 notebook.

### Why do we care?

- lsdb now enables users to search by tract/patch, as they would on the RSP
- however, using the lsst skymap requries the lsst skymap package, which depends on other packages in the lsst stack
- this means it isn't pip installable :(
- so, we provide infrastrucutre to read in these skymap files and convert them to new formats that can be read without the lsst stack

## Methods of converting and storing skymaps

There are a couple of methods we explore:

### "Full Vertex"
- this is a vertex dump
- we calculate the vertices for each tract in the skymap file, then dump into the yaml as radec values
- really has no deps which is nice
- and works for more or less any kind of skymap
- however, storage isn't ideal
- (metrics)
- plus, if we added patches, at the 10x10 patches per tract that lsst uses,
- (metrics)
- so, we use prior knowledge of the lsst's ring structure to 

### "Ring Optimized"
- `ringSkymap` files, of which the lsst skymap is a type, divide the sky into a set of horizontal "rings"
- they have consistent upper, lower dec boundaries
- they are subdivided into tracts, which (besides the poles) are all rectangular (rounding for floating point precision; conservatively beyond arcseconds)
- (could list some more invariants about rings and tracts)
- (metrics)

## Other information

There are a few more things to know about skymaps.

### Inner vs outer polygons

(describe)

### Exact tract boundaries

These are `¯\_(ツ)_/¯ `

- See Appendix 2
- todo: list info from the slack thread Neven started

## Read in raw skymap

In [ ]:
from pathlib import Path
from skymap_convert import load_pickle_skymap

package_root = Path.home() / "skymap-convert"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

In [2]:
lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

## Convert to Full Vertex skymap

Keep in mind: for inner polygons, Ring Optimized skymaps (outlined below) are the recommended type!

In [ ]:
converted_skymaps_dir = package_root / "converted_skymaps" / "full_vertex"

fv_inner_poly_path = converted_skymaps_dir / "inner_polygons.yaml"
fv_outer_poly_path = converted_skymaps_dir / "outer_polygons.yaml"

In [ ]:
from skymap_convert import write_polygons_ra_dec

write_polygons_ra_dec(lsst_skymap, fv_inner_poly_path, inner=True)
write_polygons_ra_dec(lsst_skymap, fv_outer_poly_path, inner=False)

✅ Wrote 18938 tract polygons to /sdf/home/o/olynn/skymap-convert/converted_skymaps/full_vertex/inner_polygons.yaml
✅ Wrote 18938 tract polygons to /sdf/home/o/olynn/skymap-convert/converted_skymaps/full_vertex/outer_polygons.yaml


In [ ]:
!ls -lh {fv_inner_poly_path}
!ls -lh {fv_outer_poly_path}

-rw-r--r-- 1 olynn gu 4.2M Jun 13 09:42 /sdf/home/o/olynn/skymap-convert/converted_skymaps/full_vertex/inner_polygons.yaml
-rw-r--r-- 1 olynn gu 4.3M Jun 13 09:42 /sdf/home/o/olynn/skymap-convert/converted_skymaps/full_vertex/outer_polygons.yaml


## Convert to Ring Optimized skymap

In [6]:
converted_skymaps_dir = package_root / "converted_skymaps" / "ring_optimized"

ro_inner_poly_path = converted_skymaps_dir / "inner_polygons.yaml"
ro_outer_poly_path = converted_skymaps_dir / "outer_polygons.yaml"

In [7]:
from skymap_convert import write_ring_optimized_skymap

write_ring_optimized_skymap(
    lsst_skymap, 
    ro_inner_poly_path, 
    inner=True,
    patches=False,
    skymap_name="lsst_v1"
)
# skip the outer polygons for ring optimized skymaps, for now (todo)

✅ Ring-optimized skymap written to /sdf/home/o/olynn/skymap-convert/converted_skymaps/ring_optimized/inner_polygons.yaml


In [8]:
# Use bash to see how big the file is:
!ls -lh {ro_inner_poly_path}

-rw-r--r-- 1 olynn gu 13K Jun 13 09:46 /sdf/home/o/olynn/skymap-convert/converted_skymaps/ring_optimized/inner_polygons.yaml
